# CHAPTER 10 Data Aggregation and Group Operations

这一章的内容：

* 把一个pandas对象（Series或DataFrame）按一个或多个key分解为多份（key可以是函数的形式，或者arrays, DataFrame column names)

* 计算组的汇总统计值（group summary statistics），比如计数，平均值，标准差，或用户自己定义的函数

* 应用组内的数据转换或其他一些操作，比如标准化，线性回归，排序，子集选择

* 计算透视表和交叉列表

* 进行分位数分析和其他一些统计组分析

In [1]:
import numpy as np
import pandas as pd

## 10.1 GroupBy Mechanics

Hadley Wickham，是很多有名的R语言包的作者，他描述`group operation`(组操作)为`split-apply-combine`(分割-应用-结合)。第一个阶段，存储于series或DataFrame中的数据，根据不同的keys会被split（分割）为多个组。而且分割的操作是在一个特定的axis（轴）上。例如，DataFrame能按行（axis=0）或列（axis=1）来分组。之后，我们可以把函数apply（应用）在每一个组上，产生一个新的值。最后，所以函数产生的结果被combine(结合)为一个结果对象（result object）。下面是一个图示：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/ikthz.png)

每一个用于分组的key能有很多形式，而且keys也不必都是一种类型：

* 按axis分组后含有相同长度的list或array

* DataFrame中的列名

* 一个dict或Series，给出一个对应关系，用于对应按轴分组后的值与组的名字

* 能在axis index（轴索引）或index上的labels（标签）上被调用的函数

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'], 
                   'data1' : np.random.randn(5), 
                   'data2' : np.random.randn(5)})
df

,data1,data2,key1,key2
0,0.008553,-0.507719,a,one
1,-0.319993,1.573776,a,two
2,-0.102278,1.706804,b,one
3,-0.388653,0.548088,b,two
4,0.180972,0.744596,a,one


假设我们想要，通过使用key1作为labels，来计算data1列的平均值。有很多方法可以做到这点，一种是访问data1，并且使用列（a series）在key1上，调用groupby。(译者：其实就是按key1来进行分组，但只保留data1这一列)：

In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

这个grouped变量是一个GroupBy object(分组对象)。实际上现在还没有进行任何计算，除了调用group key(分组键)df['key1']时产生的一些中间数据。整个方法是这样的，这个GroupBy object(分组对象)已经有了我们想要的信息，现在需要的是对于每一个group（组）进行一些操作。例如，通过调用GroupBy的mean方法，我们可以计算每个组的平均值：

In [7]:
grouped.mean()

key1
a   -0.087028
b    0.942529
Name: data1, dtype: float64

之后我们会对于调用.mean()后究竟发生了什么进行更详细的解释。重要的是，我们通过group key（分组键）对数据（a series）进行了聚合，这产生了一个新的Series，而且这个series的索引是key1列中不同的值。

得到的结果中，index（索引）也有'key1'，因为我们使用了df['key1']。

如果我们传入多个数组作为一个list，那么我们会得到不同的东西：

In [8]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.481997
      two     0.702910
b     one     0.186644
      two     1.698414
Name: data1, dtype: float64

这里我们用了两个key来分组，得到的结果series现在有一个多层级索引，这个多层索引是根据key1和key2不同的值来构建的：

In [9]:
means.unstack()

key2,one,two
key1,,
a,-0.481997,0.702910
b,0.186644,1.698414


在上面的例子里，group key全都是series，即DataFrame中的一列，当然，group key只要长度正确，可以是任意的数组：

In [10]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby([states, years]).mean()

California  2005    0.702910
            2006    0.186644
Ohio        2005    0.204797
            2006    0.324825
Name: data1, dtype: float64

其中分组信息经常就在我们处理的DataFrame中，在这种情况下，我们可以传入列名（可以是字符串，数字，或其他python对象）作为group keys：

In [12]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.087028,-0.000901
b,0.942529,0.503650


In [3]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.094763  0.118438
     two  -0.319993  1.573776
b    one  -0.102278  1.706804
     two  -0.388653  0.548088

我们注意到第一个例子里，df.groupby('key1').mean()的结果里并没有key2这一列。因为df['key2']这一列不是数值型数据，我们称这种列为`nuisance column`（有碍列），这种列不会出现在结果中。默认，所有的数值型列都会被汇总计算，但是出现有碍列的情况的话，就会过滤掉这种列。

一个很有用的GroupBy方法是size，会返回一个包含group size(组大小)的series：

In [4]:
df.groupby(['key1', 'key2']).size().unstack()

key2,one,two
key1,,
a,2,1
b,1,1


另外一点需要注意的是，如果作为group key的列中有缺失值的话，也不会出现在结果中。

### 1. Iterating Over Groups

GroupBy对象支持迭代，能产生一个2-tuple（二元元组），包含组名和对应的数据块。考虑下面的情况：

In [5]:
df

,data1,data2,key1,key2
0,0.008553,-0.507719,a,one
1,-0.319993,1.573776,a,two
2,-0.102278,1.706804,b,one
3,-0.388653,0.548088,b,two
4,0.180972,0.744596,a,one


In [6]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
      data1     data2 key1 key2
0  0.008553 -0.507719    a  one
1 -0.319993  1.573776    a  two
4  0.180972  0.744596    a  one
b
      data1     data2 key1 key2
2 -0.102278  1.706804    b  one
3 -0.388653  0.548088    b  two


对于有多个key的情况，元组中的第一个元素会被作为另一个元组的key值（译者：可以理解为多个key的所有组合情况）：

In [7]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
      data1     data2 key1 key2
0  0.008553 -0.507719    a  one
4  0.180972  0.744596    a  one
('a', 'two')
      data1     data2 key1 key2
1 -0.319993  1.573776    a  two
('b', 'one')
      data1     data2 key1 key2
2 -0.102278  1.706804    b  one
('b', 'two')
      data1     data2 key1 key2
3 -0.388653  0.548088    b  two


当然，也可以对数据的一部分进行各种操作。一个便利的用法是，用一个含有数据片段（data pieces）的dict来作为单行指令(one-liner)：

In [17]:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':       data1     data2 key1 key2
 0  0.008553 -0.507719    a  one
 1 -0.319993  1.573776    a  two
 4  0.180972  0.744596    a  one, 'b':       data1     data2 key1 key2
 2 -0.102278  1.706804    b  one
 3 -0.388653  0.548088    b  two}

In [19]:
pieces['b']

,data1,data2,key1,key2
2,-0.102278,1.706804,b,one
3,-0.388653,0.548088,b,two


groupby默认作用于axis=0，但是我们可以指定任意的轴。例如，我们可以按dtyple来对列进行分组：

In [20]:
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [21]:
grouped = df.groupby(df.dtypes, axis=1)

for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.008553 -0.507719
1 -0.319993  1.573776
2 -0.102278  1.706804
3 -0.388653  0.548088
4  0.180972  0.744596
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 2. Selecting a Column or Subset of Columns

如果一个GroupBy对象是由DataFrame创建来的，那么通过列名或一个包含列名的数组来对GroupBy对象进行索引的话，就相当于对列取子集做聚合（column subsetting for aggregation）。这句话的意思是：

```
df.groupby('key1')['data1'] 

df.groupby('key1')[['data2']]
```
   
上面的代码其实就是下面的语法糖（Syntactic sugar）：

    df['data1'].groupby(df['key1']) 
    df[['data2']].groupby(df['key1'])
    
`语法糖(Syntactic sugar),是由Peter J. Landin(和图灵一样的天才人物，是他最先发现了Lambda演算，由此而创立了函数式编程)创造的一个词语，它意指那些没有给计算机语言添加新功能，而只是对人类来说更“甜蜜”的语法。语法糖往往给程序员提供了更实用的编码方式，有益于更好的编码风格，更易读。不过其并没有给语言添加什么新东西。`

尤其是对于一些很大的数据集，这种用法可以聚集一部分列。例如，在处理一个数据集的时候，想要只计算data2列的平均值，并将结果返还为一个DataFrame，我们可以这样写：

In [22]:
df

,data1,data2,key1,key2
0,0.008553,-0.507719,a,one
1,-0.319993,1.573776,a,two
2,-0.102278,1.706804,b,one
3,-0.388653,0.548088,b,two
4,0.180972,0.744596,a,one


In [23]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.118438
     two   1.573776
b    one   1.706804
     two   0.548088

In [24]:
s_grouped = df.groupby(['key1', 'key2'])[['data1', 'data2']]
s_grouped

In [25]:
s_grouped.mean()

data1     data2
key1 key2                    
a    one   0.094763  0.118438
     two  -0.319993  1.573776
b    one  -0.102278  1.706804
     two  -0.388653  0.548088

### 3. Grouping with Dicts and Series

分组信息可以不是数组的形式。考虑下面的例子：

In [26]:
people = pd.DataFrame(np.random.randn(5,5), 
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,-0.062008,1.250052,-0.075907,-0.464687,-0.566399
Steve,-0.843778,1.538773,-0.351484,-0.065239,-0.021753
Wes,-0.912797,NaN,NaN,0.407573,0.404426
Jim,-1.319795,-0.960031,0.355724,0.684700,-0.543182
Travis,0.310037,1.419185,0.484242,0.548955,1.723964


假设我们有一个组，对应多个列，而且我们想要按组把这些列的和计算出来：

In [27]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f': 'orange'}

现在，我们可以通过这个dict构建一个数组，然后传递给groupby，但其实我们可以直接传入dict（可以注意到key里有一个'f'，这说明即使有，没有被用到的group key，也是ok的）：

In [28]:
by_columns = people.groupby(mapping, axis=1)
by_columns.sum()

,blue,red
Joe,-0.540595,0.621646
Steve,-0.416723,0.673242
Wes,0.407573,-0.508371
Jim,1.040424,-2.823008
Travis,1.033197,3.453186


这种用法同样适用于series，这种情况可以看作是固定大小的映射（fixed-size mapping）:

In [30]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [31]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [32]:
people.groupby(map_series, axis=1).size()

blue    2
red     3
dtype: int64

### 4. Grouping with Functions

比起用dict或series定义映射关系，使用python的函数是更通用的方法。任何一个作为group key的函数，在每一个index value（索引值）上都会被调用一次，函数计算的结果在返回的结果中会被用做group name。更具体一点，考虑前一个部分的DataFrame，用人的名字作为索引值。假设我们想要按照名字的长度来分组；同时我们要计算字符串的长度，使用len函数会变得非常简单：

In [33]:
people.groupby(len).sum()
# len函数在每一个index（即名字）上被调用了

,a,b,c,d,e
3,-2.294600,0.290022,0.279817,0.627586,-0.705154
5,-0.843778,1.538773,-0.351484,-0.065239,-0.021753
6,0.310037,1.419185,0.484242,0.548955,1.723964


混合不同的函数、数组，字典或series都不成问题，因为所有对象都会被转换为数组：

In [40]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.912797  1.250052 -0.075907 -0.464687 -0.566399
  two -1.319795 -0.960031  0.355724  0.684700 -0.543182
5 one -0.843778  1.538773 -0.351484 -0.065239 -0.021753
6 two  0.310037  1.419185  0.484242  0.548955  1.723964

### 5. Grouping by Index Levels

最后关于多层级索引数据集(hierarchically indexed dataset)，一个很方便的用时是在聚集（aggregate）的时候，使用轴索引的层级（One of the levels of an axis index）。看下面的例子：

In [41]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'], 
                                     [1, 3, 5, 1, 3]], 
                                    names=['cty', 'tenor'])
columns

MultiIndex(levels=[['JP', 'US'], [1, 3, 5]],
           labels=[[1, 1, 1, 0, 0], [0, 1, 2, 0, 1]],
           names=['cty', 'tenor'])

In [42]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.469780  0.638290  0.227915  1.002858  2.315825
1      0.563677 -0.654445 -3.575667 -0.410529  0.599758
2      0.769956  0.177737  0.982014 -0.441651  0.669867
3     -0.369750 -0.244317  0.166514 -0.873611 -0.375791

要想按层级分组，传入层级的数字或者名字，通过使用level关键字：

In [43]:
hier_df.groupby(level='cty',axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 Data Aggregation

聚合（Aggregation）指能从数组(array)中产生标量（scalar values）的数据转化（data transformation）。下面的例子就是一些聚合方法，包括mean, count, min and sum。我们可能会好奇，在一个GroupBy对象上调用mean()的时候，究竟发生了什么。一些常见的聚合，比如下表，实现方法上都已经被优化过了。当然，我们可以使用的聚合方法不止这些：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/sugsj.png)

我们可以使用自己设计的聚合方法，而且可以调用分组后对象上的任意方法。例如，我们可以调用quantile来计算Series或DataFrame中列的样本的百分数。

尽管quantile并不是专门为GroupBy对象设计的方法，这是一个Series方法，但仍可以被GroupBy对象使用。GroupBy会对Series进行切片（slice up），并对于切片后的每一部分调用piece.quantile(0.9)，然后把每部分的结果整合到一起：

In [11]:
df = pd.DataFrame({
    'key1':['a', 'a', 'b', 'b', 'a'],
    'key2':['one', 'two', 'one', 'two', 'one'], 
    'data1':np.random.randn(5),
    'data2':np.random.randn(5)
})

df

,data1,data2,key1,key2
0,1.202162,-0.179657,a,one
1,0.095677,-1.336717,a,two
2,-1.229161,-0.298188,b,one
3,0.685477,0.043629,b,two
4,0.063797,-0.579232,a,one


In [12]:
grouped = df.groupby('key1')
for key, group in grouped:
    print(key)
    print(group)

a
      data1     data2 key1 key2
0  1.202162 -0.179657    a  one
1  0.095677 -1.336717    a  two
4  0.063797 -0.579232    a  one
b
      data1     data2 key1 key2
2 -1.229161 -0.298188    b  one
3  0.685477  0.043629    b  two


In [14]:
'''
确定p分位数的位置
pos = 1+(n-1)*p

然后再计算相应的值
比如上面a
pos = 1 + (3-1) * 0.9 = 2.8
result = 0.095677 + (1.202162 - 0.095677) * 0.8 = 0.980865
'''
grouped['data1'].quantile(0.9)

key1
a    0.980865
b    0.494014
Name: data1, dtype: float64

如果想用自己设计的聚合函数，把用于聚合数组的函数传入到aggregate或agg方法即可：

In [16]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.138366,1.157060
b,1.914638,0.341817


我们发现很多方法，比如describe，也能正常使用，尽管严格的来说，这并不是聚合：

In [22]:
grouped.describe().stack()

data1     data2
key1                          
a    count  3.000000  3.000000
     mean   0.453879 -0.698535
     std    0.648229  0.587684
     min    0.063797 -1.336717
     25%    0.079737 -0.957975
     50%    0.095677 -0.579232
     75%    0.648920 -0.379445
     max    1.202162 -0.179657
b    count  2.000000  2.000000
     mean  -0.271842 -0.127280
     std    1.353854  0.241701
     min   -1.229161 -0.298188
     25%   -0.750501 -0.212734
     50%   -0.271842 -0.127280
     75%    0.206818 -0.041826
     max    0.685477  0.043629

细节的部分在10.3会进行更多解释。

注意：自定义的函数会比上面表中的函数慢一些，上面的函数时优化过的，而自定义的函数会有一些额外的计算，所以慢一些。

### 1. Column-Wise and Multiple Function Application

让我们回到tipping数据集。加载数据及后，我们添加一列用于描述小费的百分比：

In [25]:
tips = pd.read_csv("./Data/Example/tips.csv")

# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


我们可以看到，对series或DataFrame进行聚合，其实就是通过aggregate使用合适的函数，或者调用一些像mean或std这样的方法。然而，我们可能想要在列上使用不同的函数进行聚合，又或者想要一次执行多个函数。幸运的是，这是可能的，下面将通过一些例子来说明。首先，对于tips数据集，先用day和smoker进行分组：

In [27]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
for key, group in grouped_pct:
    print(key)
    print(group)

('Fri', 'No')
91     0.155625
94     0.142857
99     0.120385
223    0.187735
Name: tip_pct, dtype: float64
('Fri', 'Yes')
90     0.103555
92     0.173913
93     0.263480
95     0.117750
96     0.146628
97     0.124688
98     0.142789
100    0.220264
101    0.195059
220    0.180921
221    0.259314
222    0.223776
224    0.117735
225    0.153657
226    0.198216
Name: tip_pct, dtype: float64
('Sat', 'No')
19     0.162228
20     0.227679
21     0.135535
22     0.141408
23     0.192288
24     0.160444
25     0.131387
26     0.149589
27     0.157604
28     0.198157
29     0.152672
30     0.151832
31     0.136240
32     0.199203
33     0.118415
34     0.183915
35     0.149626
36     0.122624
37     0.181335
38     0.123596
39     0.159898
40     0.139651
57     0.056797
59     0.139424
64     0.150085
65     0.156873
66     0.150152
68     0.099357
70     0.163894
71     0.175747
74     0.149355
75     0.118934
104    0.195029
108    0.206140
110    0.214286
111    0.137931
212    0.186220
2

对于像是上面表格10-1中的一些描述性统计，我们可以直接传入函数的名字，即字符串：

In [28]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

如果我们把函数或函数的名字作为一个list传入，我们会得到一个DataFrame，每列的名字就是函数的名字：

In [29]:
# def peak_to_peak(arr):
#     return arr.max() - arr.min()
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

上面我们把多个聚合函数作为一个list传入给agg，这些函数会独立对每一个组进行计算。

上面结果的列名是自动给出的，当然，我们也可以更改这些列名。这种情况下，传入一个由tuple组成的list，每个tuple的格式是(name, function)，每个元组的第一个元素会被用于作为DataFrame的列名（我们可以认为这个二元元组list是一个有序的映射）：

In [30]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

如果是处理DataFrame，我们有更多的选择，我们可以用一个含有多个函数的list应用到所有的列上，也可以在不同的列上应用不同的函数。演示一下，假设我们想要在tip_pct和total_bill这两列上，计算三个相同的统计指标：

In [31]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

我们可以看到，结果中的DataFrame有多层级的列（hierarchical columns）。另外一种做法有相同的效果，即我们对于每一列单独进行聚合（aggregating each column separately），然后使用concat把结果都结合在一起，然后用列名作为keys参数：

In [32]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

我们之前提到过，可以用元组组成的list来自己定义列名：

In [33]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

现在，假设我们想要把不同的函数用到一列或多列上。要做到这一点，给agg传递一个dict，这个dict需要包含映射关系，用来表示列名和函数之间的对应关系：

In [34]:
grouped.agg({'tip':np.max, 'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [35]:
grouped.agg({'tip_pct':['min', 'max', 'mean', 'std'], 'size':'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

只有当多个函数用于至少一列的时候，DataFrame才会有多层级列（hierarchical columns）

### 2. Returning Aggregated Data Without Row Indexes

目前为止提到的所有例子，最后返回的聚合数据都是有索引的，而且这个索引默认是多层级索引，这个索引是由不同的组键的组合构成的（unique group key combinations）。因为我们并不是总需要返回这种索引，所以我们可以取消这种模式，在调用groupby的时候设定as_index=False即可：

In [36]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 10.3 Apply：General split-apply-combine

> general-purpose: 可以理解为通用，泛用。

>例子：在计算机软件中，通用编程语言(General-purpose programming language )指被设计为各种应用领域服务的编程语言。通常通用编程语言不含有为特定应用领域设计的结构。

>相对而言，特定域编程语言就是为某一个特定的领域或应用软件设计的编程语言。比如说，LaTeX就是专门为排版文献而设计的语言。

最通用的GroupBy(分组)方法是apply，这也是本节的主题。如下图所示，apply会把对象分为多个部分，然后将函数应用到每一个部分上，然后把所有的部分都合并起来：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/81f9f.png)

返回之前提到的tipping数据集，假设我们想要根据不同组（group），选择前5个tip_pct值最大的。首先，写一个函数，函数的功能为在特定的列，选出有最大值的行:

In [2]:
tips = pd.read_csv("./Data/Example/tips.csv")

# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [3]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


现在，如果我们按smoker分组，然后用apply来使用这个函数，我们能得到下面的结果：

In [4]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

我们来解释下上面这一行代码发生了什么。这里的top函数，在每一个DataFrame中的行组（row group）都被调用了一次，然后各自的结果通过pandas.concat合并了，最后用组名（group names）来标记每一部分。（译者：可以理解为，我们先按smoker这一列对整个DataFrame进行了分组，一共有No和Yes两组，然后对每一组上调用了top函数，所以每一组会返还5行作为结果，最后把两组的结果整合起来，一共是10行）。

最后的结果是有多层级索引（hierarchical index）的，而且这个多层级索引的内部层级（inner level）含有来自于原来DataFrame中的索引值（index values）（译者：即在smoker为No的这一组，No本身是一个索引，它的内层索引是88, 185, 51, 149, 232这五个行索引，这五个内部层级是来自于原始DataFrame的）。

如果传递一个函数给apply，可以在函数之后，设定其他一些参数：

In [5]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column="total_bill")

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

除了上面这些基本用法，要想用好apply可能需要一点创新能力。毕竟传给这个函数的内容取决于我们自己，而最终的结果只需要返回一个pandas对象或一个标量。这一章的剩余部分主要介绍如何解决在使用groupby时遇到的一些问题。

可以试一试在GroupBy对象上调用describe

In [9]:
result = tips.groupby('smoker')['tip_pct'].describe()

result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [19]:
result.stack().unstack('smoker')

smoker,No,Yes
count,151.000000,93.000000
mean,0.159328,0.163196
std,0.039910,0.085119
min,0.056797,0.035638
25%,0.136906,0.106771
50%,0.155625,0.153846
75%,0.185014,0.195059
max,0.291990,0.710345


在GroupBy内部，当我们想要调用一个像describe这样的函数的时候，其实相当于下面的写法：

    f = lambda x: x.describe()
    grouped.apply(f)

### 1. Suppressing the Group Keys

在接下来的例子，我们会看到作为结果的对象有一个多层级索引（hierarchical index），这个多层级索引是由原来的对象中，组键（group key）在每一部分的索引上得到的。我们可以在groupby函数中设置group_keys=False来关闭这个功能：

In [21]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### 2. Quantile and Bucket Analysis

在第八章中，我们介绍了pandas的一些工具，比如cut和qcut，通过设置中位数，切割数据为buckets with bins(有很多箱子的桶)。

> 这里bucket我翻译为桶，可以理解为像group一样的概念，一个组内有不同的bins。而关于bins（箱）的部分，可以回顾看一下7.2

把函数通过groupby整合起来，可以在做桶分析或分位数分析的时候更方便。假设一个简单的随机数据集和一个等长的桶类型（bucket categorization），使用cut： 

In [2]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                       'data2': np.random.randn(1000)})
frame.head()

,data1,data2
0,-1.117411,-1.714515
1,-0.544665,-0.203708
2,0.800810,-1.256846
3,-0.521751,0.109719
4,-0.664808,-0.659305


In [3]:
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0    (-1.611, -0.121]
1    (-1.611, -0.121]
2      (-0.121, 1.37]
3    (-1.611, -0.121]
4    (-1.611, -0.121]
5      (-0.121, 1.37]
6        (1.37, 2.86]
7      (-0.121, 1.37]
8      (-0.121, 1.37]
9      (-0.121, 1.37]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.107, -1.611] < (-1.611, -0.121] < (-0.121, 1.37] < (1.37, 2.86]]

cut返回的Categorical object（类别对象）能直接传入groupby。所以我们可以在data2列上计算很多统计值：

In [5]:
def get_stats(group):
    return {'min':group.min(), 'max':group.max(),
           'count':group.count(), 'mean':group.mean()}

grouped = frame.data2.groupby(quartiles)

grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.107, -1.611]",50.0,2.335169,0.080644,-2.547794
"(-1.611, -0.121]",400.0,3.529765,-0.042367,-2.630203
"(-0.121, 1.37]",449.0,2.822909,-0.031766,-2.756518
"(1.37, 2.86]",101.0,2.489412,0.006706,-2.550062


也有相同长度的桶（equal-length buckets）；想要按照样本的分位数得到相同长度的桶，用qcut。这里设定labels=False来得到分位数的数量：

In [11]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)

grouped = frame.data2.groupby(grouping)

result = grouped.apply(get_stats).unstack()
result

,count,max,mean,min
data1,,,,
0,100.0,2.335169,0.053026,-2.547794
1,100.0,2.178563,-0.091840,-2.630203
2,100.0,3.529765,0.051323,-2.319132
3,100.0,1.948993,-0.168303,-2.325652
4,100.0,2.450706,-0.011882,-2.378696
5,100.0,2.335652,-0.012119,-2.602760
6,100.0,2.822909,-0.023510,-2.220064
7,100.0,2.384273,0.003391,-2.756518
8,100.0,2.206631,-0.080611,-2.281385


译者：上面的代码是把frame的data1列分为10个bin，每个bin都有相同的数量。因为一共有1000个样本，所以每个bin里有100个样本。grouping保存的是每个样本的index以及其对应的bin的编号。

对于pandas的Categorical类型，会在第十二章做详细介绍。


### 3 Example: Filling Missing Values with Group-Specific Values

在处理缺失值的时候，一些情况下我们会直接用dropna来把缺失值删除，但另一些情况下，我们希望用一些固定的值来代替缺失值，而fillna就是用来做这个的，例如，这里我们用平均值mean来代替缺失值NA：

In [12]:
s = pd.Series(np.random.randn(5))
s[::2] = np.nan

s

0         NaN
1    0.158210
2         NaN
3    0.058141
4         NaN
dtype: float64

In [13]:
s.fillna(s.mean())

0    0.108175
1    0.158210
2    0.108175
3    0.058141
4    0.108175
dtype: float64

假设我们想要给每一组填充不同的值。一个方法就是对数据分组后，用apply来调用fillna，在每一个组上执行一次。这里有一些样本是把美国各州分为西部和东部：

In [20]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']

group_keys = ['East'] * 4 + ['West'] * 4
group_keys

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [21]:
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -0.210587
New York      1.283387
Vermont      -1.689923
Florida      -2.883676
Oregon       -0.423789
Nevada       -0.787763
California   -2.030747
Idaho         0.958587
dtype: float64

In [22]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -0.210587
New York      1.283387
Vermont            NaN
Florida      -2.883676
Oregon       -0.423789
Nevada             NaN
California   -2.030747
Idaho              NaN
dtype: float64

In [23]:
fill_mean = lambda g:g.fillna(g.mean())

data.groupby(group_keys).apply(fill_mean)

Ohio         -0.210587
New York      1.283387
Vermont      -0.603625
Florida      -2.883676
Oregon       -0.423789
Nevada       -1.227268
California   -2.030747
Idaho        -1.227268
dtype: float64

在另外一些情况下，我们可能希望提前设定好用于不同组的填充值。因为group有一个name属性，我们可以利用这个：

In [27]:
fill_values = {'East':0.5, 'West':0.1}
fill_func = lambda g:g.fillna(fill_values[g.name])

data.groupby(group_keys).apply(fill_func)

Ohio         -0.210587
New York      1.283387
Vermont       0.500000
Florida      -2.883676
Oregon       -0.423789
Nevada        0.100000
California   -2.030747
Idaho         0.100000
dtype: float64

### 4 Example: Random Sampling and Permutation

假设我们想要从一个很大的数据集里随机抽出一些样本，这里我们可以在Series上用sample方法。为了演示，这里创建一副模拟的扑克牌：

In [28]:
# Hearts红桃，Spades黑桃，Clubs梅花，Diamonds方片
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []

for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

这样我们就得到了一个长度为52的Series，索引（index）部分是牌的名字，对应的值为牌的点数，这里的点数是按Blackjack（二十一点）的游戏规则来设定的。

> Blackjack（二十一点）: 2点至10点的牌以牌面的点数计算，J、Q、K 每张为10点，A可记为1点或为11点。这里为了方便，我们只把A记为1点。

In [29]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

现在，就像我们上面说的，随机从牌组中抽出5张牌：

In [30]:
def draw(deck, n=5):
    return deck.sample(n)

draw(deck)

8C    8
2H    2
6C    6
AD    1
2D    2
dtype: int64

假设我们想要从每副花色中随机抽取两张，花色是每张牌名字的最后一个字符（即H, S, C, D），我们可以根据花色分组，然后使用apply：

In [31]:
# last letter is suit
get_suit = lambda card: card[-1]
deck.groupby(get_suit).apply(draw, n=2)

C  AC     1
   7C     7
D  QD    10
   2D     2
H  3H     3
   6H     6
S  QS    10
   9S     9
dtype: int64

另外一种写法：

In [32]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

JC    10
AC     1
QD    10
5D     5
3H     3
AH     1
6S     6
7S     7
dtype: int64

### 5 Example: Group Weighted Average and Correlation

在groupby的split-apply-combine机制下，DataFrame的两列或两个Series，计算组加权平均（Group Weighted Average）是可能的。这里举个例子，下面的数据集包含组键，值，以及权重：

In [4]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,1.582062,0.111111
1,a,-0.354054,0.244409
2,a,2.887652,0.959308
3,a,-0.234869,0.084489
4,b,0.250839,0.284906
5,b,-0.339405,0.432754
6,b,0.634681,0.455488
7,b,1.386976,0.559162


按category分组来计算组加权平均：

In [5]:
grouped = df.groupby('category')
get_wavg = lambda g:np.average(g["data"], weights=g['weights'])

grouped.apply(get_wavg)

category
a    2.029243
b    0.571041
dtype: float64

另一个例子，考虑一个从Yahoo！财经上得到的经济数据集，包含一些股票交易日结束时的股价，以及S&P 500指数(即SPX符号)：

> 标准普尔500指数英文简写为S&P 500 Index，是记录美国500家上市公司的一个股票指数。这个股票指数由标准普尔公司创建并维护。

> 标准普尔500指数覆盖的所有公司，都是在美国主要交易所，如纽约证券交易所、Nasdaq交易的上市公司。与道琼斯指数相比，标准普尔500指数包含的公司更多，因此风险更为分散，能够反映更广泛的市场变化。

In [6]:
close_px = pd.read_csv('./Data/Example/stock_px_2.csv', parse_dates=True, index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [7]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


一个比较有意思的尝试是计算一个DataFrame，包括与SPX这一列逐年日收益的相关性（计算百分比变化）。一个可能的方法是，我们先创建一个能计算不同列相关性的函数，然后拿每一列与SPX这一列求相关性：

In [8]:
spx_corr = lambda x:x.corrwith(x['SPX'])

然后我们通过pct_change在close_px上计算百分比的变化:

In [9]:
rets = close_px.pct_change().dropna()

最后，我们按年来给这些百分比变化分组，年份可以从每行的标签中通过一个一行函数提取，然后返回的结果中，用datetime标签来表示年份：

In [10]:
get_year = lambda x:x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


我们也可以计算列内的相关性。这里我们计算苹果和微软每年的相关性：

In [41]:
by_year.apply(lambda g:g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### 6 Example: Group-Wise Linear Regression

就像上面介绍的例子，使用groupby可以用于更复杂的组对组统计分析，只要函数能返回一个pandas对象或标量。例如，我们可以定义regress函数（利用statsmodels库），在每一个数据块（each chunk of data）上进行普通最小平方回归（ordinary least squares (OLS) regression）计算：

In [12]:
import statsmodels.api as sm

def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y, X).fit()
    return result.params

In [13]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 10.4 Pivot Tables and Cross-Tabulation

Pivot Tables（数据透视表）是一种常见的数据汇总工具，常见与各种spreadsheet programs（电子表格程序，比如Excel）和一些数据分析软件。它能按一个或多个keys来把数据聚合为表格，能沿着行或列，根据组键来整理数据。

数据透视表可以用pandas的groupby来制作，这个本节会进行介绍，除此之外还会有介绍如何利用多层级索引来进行reshape（更改形状）操作。DataFrame有一个pivot_table方法，另外还有一个pandas.pivot_table函数。为了有一个更方便的groupby接口，pivot_table能添加partial totals（部分合计）,也被称作margins(边界)。

回到之前提到的tipping数据集，假设我们想要计算一个含有组平均值的表格(a table of group means，这个平均值也是pivot_table默认的聚合类型)，按day和smoker来分组：

In [15]:
tips = pd.read_csv('./Data/Example/tips.csv')
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [16]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

这个结果也可以通过groupby直接得到。

现在假设我们想要按time分组，然后对tip_pct和size进行聚合。我们会把smoker放在列上，而day用于行：

In [18]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

我们也快成把这个表格加强一下，通过设置margins=True来添加部分合计（partial total）。这么做的话有一个效果，会给行和列各添加All标签，这个All表示的是当前组对于整个数据的统计值：

In [19]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

这里，对于All列，这一列的值是不考虑吸烟周和非吸烟者的平均值（smoker versus nonsmoker）。对于All行，这一行的值是不考虑任何组中任意两个组的平均值（any of the two levels of grouping）。

想要使用不同的聚合函数，传递给aggfunc即可。例如，count或len可以给我们一个关于组大小（group size）的交叉表格：

In [20]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

如果一些组合是空的（或NA），我们希望直接用fill_value来填充：

In [21]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

下面是关于pivot_table方法的一些选项：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/doyxv.png)

### Cross-Tabulations: Crosstab

cross-tabulation（交叉表，简写为crosstab），是数据透视表的一个特殊形式，只计算组频率（group frequencies）。这里有个例子：

In [22]:
data = pd.DataFrame({'Sample': np.arange(1, 11),
        'Nationality': ['USA', 'Japan', 'USA', 'Japan', 'Japan', 'Japan', 'USA', 'USA', 'Japan', 'USA'],
        'Handedness': ['Right-handed', 'Left-handed', 'Right-handed', 
                       'Right-handed', 'Left-handed', 'Right-handed', 
                       'Right-handed', 'Left-handed', 'Right-handed', 'Right-handed']})
data

,Handedness,Nationality,Sample
0,Right-handed,USA,1
1,Left-handed,Japan,2
2,Right-handed,USA,3
3,Right-handed,Japan,4
4,Left-handed,Japan,5
5,Right-handed,Japan,6
6,Right-handed,USA,7
7,Left-handed,USA,8
8,Right-handed,Japan,9
9,Right-handed,USA,10


作为调查分析（survey analysis）的一部分，我们想要按国家和惯用手来进行汇总。我们可以使用pivot_table来做到这点，不过pandas.crosstab函数会更方便一些：

In [23]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


crosstab的前两个参数可以是数组或Series或由数组组成的列表（a list of array）。对于tips数据，可以这么写：

In [24]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244